# 「PyTorchモジュールのプロファイリング方法」

【原題】Profiling your PyTorch Module

【原著】[Suraj Subramanian](https://github.com/suraj813)

【元URL】https://pytorch.org/tutorials/beginner/profiler.html

【翻訳】電通国際情報サービスISID HCM事業部　櫻井 亮佑

【日付】2020年1月28日

【チュトーリアル概要】

PyTorchには、PyTorchの様々な処理を行うコードの、実行に掛かる時間、および、メモリのコストを特定するために役に立つ、プロファイラーAPIが存在します。

プロファイラーは簡単にコードに組み込むことができ、結果を表形式で出力したり、JSON形式のトレースファイルとして返すことが可能です。

  


**注釈**<br>
プロファイラーはマルチスレッドのモデルをサポートしています。
プロファイラーは処理と同じスレッド内で実行されますが、別のスレッドで実行される可能性のある子の処理もプロファイルします。
ただし、同時に実行されるプロファイラーは、結果が混在することを防ぐために自身のスレッド内に収まるようになっています。


**注釈**<br>
PyTorch 1.8では新たなAPIを導入し、今後のリリースで旧式のプロファイラーAPIを置き換えていきます。
新しいAPIについては、[こちらのページ](https://pytorch.org/docs/master/profiler.html)をご参照ください。

プロファイラーAPIの使用方法について手短に確認したい場合は、[こちらのレシピ](https://pytorch.org/tutorials/recipes/recipes/profiler.html)をご参照ください。

--------------



In [ ]:
import torch
import numpy as np
from torch import nn
import torch.autograd.profiler as profiler

In [ ]:
%matplotlib inline

## プロファイラーを用いたパフォーマンスのデバッグ

プロファイラーは、モデル内のパファーマンスのボトルネックを特定する上で役立ちます。
本チュートリアルでは例として、以下の2つのサブタスクをこなすオリジナルのモジュールを構築します。

- 入力の線形変換
- 線形変換の結果を用いたマスクテンソルのインデックスの取得

`profiler.record_function("label")`を使用し、各サブタスクを行うコードを分離して、ラベル付けされたコンテクストマネージャー内にラップします。

プロファイラーの出力では、サブタスク内のすべての処理の総合的なパフォーマンス指標が、対応するラベルの下に表示されます。

なお、プロファイラーを使用することにより、オーバーヘッドが多少生じます。そのため、プロファイラーはコードを調査する目的のみに使用することが最善です。

実行時間のベンチマークを測定する際は、プロファイラーの部分を取り除くことを忘れないでおきましょう。

In [ ]:
class MyModule(nn.Module):
    def __init__(self, in_features: int, out_features: int, bias: bool = True):
        super(MyModule, self).__init__()
        self.linear = nn.Linear(in_features, out_features, bias)

    def forward(self, input, mask):
        with profiler.record_function("LINEAR PASS"):
            out = self.linear(input)

        with profiler.record_function("MASK INDICES"):
            threshold = out.sum(axis=1).mean().item()
            hi_idx = np.argwhere(mask.cpu().numpy() > threshold)
            hi_idx = torch.from_numpy(hi_idx).cuda()

        return out, hi_idx

## フォワードパスのプロファイル

ランダム入力とマスクテンソル、そしてモデルを初期化します。

プロファイラーを実行する前に、正確なパフォーマンスのベンチマークを測定するためにCUDAの準備をします。

`profiler.profile`コンテクストマネージャー内でモジュールのフォワードパスをラップします。

`with_stack=True`パラメーターは、ファイルとトレース内の処理の行数を追加で記録します。

**注意**<br>
`with_stack=True`はオーバーヘッドを生じるため、コードを調査する目的にのみ使用します。
。
パフォーマンスのベンチマーク測定をする際は、`with_stack=False`にすることを忘れないようにしましょう。

In [ ]:
model = MyModule(500, 10).cuda()
input = torch.rand(128, 500).cuda()
mask = torch.rand((500, 500, 500), dtype=torch.double).cuda()

# warm-up
model(input, mask)

with profiler.profile(with_stack=True, profile_memory=True) as prof:
    out, idx = model(input, mask)

## プロファイル結果の出力

最後にプロファイル結果を出力します。

`profiler.key_averages`で処理名、任意で入力の形状、またはスタックトレースのイベント毎で結果を集約します。

入力の形状でグループ化を行うことは、どのテンソルの形状がモデルに利用されているか特定する上で役に立ちます。

ここで、処理と（直近5つのイベントのみに切り取られた）処理のトレースバック毎に実行時間を集約する`group_by_stack_n=5` を使用し、それらが登録された順番でイベントを表示します。

なお表は、`sort_by`引数を与えることでソートできます。有効なソートキーについては、[ドキュメント](https://pytorch.org/docs/stable/autograd.html#profiler)をご参照ください。

**注釈**<br>
ノートブック内でプロファイラーを実行していると、スタックトレース内のファイル名ではなく、`<ipython-input-18-193a910735e8>(13): forward`のようなエントリを目にするかもしれません。
これらは、`<notebook-cell>(line number): calling-function`に対応しています。

In [ ]:
print(prof.key_averages(group_by_stack_n=5).table(sort_by='self_cpu_time_total', row_limit=5))

"""
(Some columns are omitted)

-------------  ------------  ------------  ------------  ---------------------------------
         Name    Self CPU %      Self CPU  Self CPU Mem   Source Location
-------------  ------------  ------------  ------------  ---------------------------------
 MASK INDICES        87.88%        5.212s    -953.67 Mb  /mnt/xarfuse/.../torch/au
                                                         <ipython-input-...>(10): forward
                                                         /mnt/xarfuse/.../torch/nn
                                                         <ipython-input-...>(9): <module>
                                                         /mnt/xarfuse/.../IPython/

  aten::copy_        12.07%     715.848ms           0 b  <ipython-input-...>(12): forward
                                                         /mnt/xarfuse/.../torch/nn
                                                         <ipython-input-...>(9): <module>
                                                         /mnt/xarfuse/.../IPython/
                                                         /mnt/xarfuse/.../IPython/

  LINEAR PASS         0.01%     350.151us         -20 b  /mnt/xarfuse/.../torch/au
                                                         <ipython-input-...>(7): forward
                                                         /mnt/xarfuse/.../torch/nn
                                                         <ipython-input-...>(9): <module>
                                                         /mnt/xarfuse/.../IPython/

  aten::addmm         0.00%     293.342us           0 b  /mnt/xarfuse/.../torch/nn
                                                         /mnt/xarfuse/.../torch/nn
                                                         /mnt/xarfuse/.../torch/nn
                                                         <ipython-input-...>(8): forward
                                                         /mnt/xarfuse/.../torch/nn

   aten::mean         0.00%     235.095us           0 b  <ipython-input-...>(11): forward
                                                         /mnt/xarfuse/.../torch/nn
                                                         <ipython-input-...>(9): <module>
                                                         /mnt/xarfuse/.../IPython/
                                                         /mnt/xarfuse/.../IPython/

-----------------------------  ------------  ---------- ----------------------------------
Self CPU time total: 5.931s

"""

-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
                         Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  Source Location                                                              
-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
                 MASK INDICES        81.50%        3.702s        99.93%        4.539s        4.539s          -4 b    -953.67 Mb       2.79 Gb      -1.00 Kb             1  /usr/local/lib/python3.6/dist-packages/torch/autograd/profiler.py(503): __e  
   

'\n(Some columns are omitted)\n\n-------------  ------------  ------------  ------------  ---------------------------------\n         Name    Self CPU %      Self CPU  Self CPU Mem   Source Location\n-------------  ------------  ------------  ------------  ---------------------------------\n MASK INDICES        87.88%        5.212s    -953.67 Mb  /mnt/xarfuse/.../torch/au\n                                                         <ipython-input-...>(10): forward\n                                                         /mnt/xarfuse/.../torch/nn\n                                                         <ipython-input-...>(9): <module>\n                                                         /mnt/xarfuse/.../IPython/\n\n  aten::copy_        12.07%     715.848ms           0 b  <ipython-input-...>(12): forward\n                                                         /mnt/xarfuse/.../torch/nn\n                                                         <ipython-input-...>(9): <module>\n      

## メモリパフォーマンスの向上

メモリと処理時間の観点から、最も高コストな処理は、マスクインデックス内の処理``forward (10)``である点に留意してください。

まずはメモリ消費の改善について取り組んでみましょう。

12行目の``.to()``で、953.67 Mbを消費していることがわかります（※日本語版注釈：``.cpu()``を指すと思われます）。

この処理は、``mask``をCPUにコピーしています。

そして``mask``は、``torch.double``型で初期化されています。

``torch.double``型を``torch.float``型にキャストすることで、メモリの負荷を低減できるでしょうか。

In [ ]:
model = MyModule(500, 10).cuda()
input = torch.rand(128, 500).cuda()
mask = torch.rand((500, 500, 500), dtype=torch.float).cuda()

# 準備
model(input, mask)

with profiler.profile(with_stack=True, profile_memory=True) as prof:
    out, idx = model(input, mask)

print(prof.key_averages(group_by_stack_n=5).table(sort_by='self_cpu_time_total', row_limit=5))

"""
(Some columns are omitted)

-----------------  ------------  ------------  ------------  --------------------------------
             Name    Self CPU %      Self CPU  Self CPU Mem   Source Location
-----------------  ------------  ------------  ------------  --------------------------------
     MASK INDICES        93.61%        5.006s    -476.84 Mb  /mnt/xarfuse/.../torch/au
                                                             <ipython-input-...>(10): forward
                                                             /mnt/xarfuse/  /torch/nn
                                                             <ipython-input-...>(9): <module>
                                                             /mnt/xarfuse/.../IPython/

      aten::copy_         6.34%     338.759ms           0 b  <ipython-input-...>(12): forward
                                                             /mnt/xarfuse/.../torch/nn
                                                             <ipython-input-...>(9): <module>
                                                             /mnt/xarfuse/.../IPython/
                                                             /mnt/xarfuse/.../IPython/

 aten::as_strided         0.01%     281.808us           0 b  <ipython-input-...>(11): forward
                                                             /mnt/xarfuse/.../torch/nn
                                                             <ipython-input-...>(9): <module>
                                                             /mnt/xarfuse/.../IPython/
                                                             /mnt/xarfuse/.../IPython/

      aten::addmm         0.01%     275.721us           0 b  /mnt/xarfuse/.../torch/nn
                                                             /mnt/xarfuse/.../torch/nn
                                                             /mnt/xarfuse/.../torch/nn
                                                             <ipython-input-...>(8): forward
                                                             /mnt/xarfuse/.../torch/nn

      aten::_local        0.01%     268.650us           0 b  <ipython-input-...>(11): forward
      _scalar_dense                                          /mnt/xarfuse/.../torch/nn
                                                             <ipython-input-...>(9): <module>
                                                             /mnt/xarfuse/.../IPython/
                                                             /mnt/xarfuse/.../IPython/

-----------------  ------------  ------------  ------------  --------------------------------
Self CPU time total: 5.347s

"""

-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
                         Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  Source Location                                                              
-----------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
                 MASK INDICES        71.65%        2.312s        99.96%        3.225s        3.225s          -4 b    -476.84 Mb       2.57 Gb      -1.00 Kb             1  /usr/local/lib/python3.6/dist-packages/torch/autograd/profiler.py(503): __e  
   

'\n(Some columns are omitted)\n\n-----------------  ------------  ------------  ------------  --------------------------------\n             Name    Self CPU %      Self CPU  Self CPU Mem   Source Location\n-----------------  ------------  ------------  ------------  --------------------------------\n     MASK INDICES        93.61%        5.006s    -476.84 Mb  /mnt/xarfuse/.../torch/au\n                                                             <ipython-input-...>(10): forward\n                                                             /mnt/xarfuse/  /torch/nn\n                                                             <ipython-input-...>(9): <module>\n                                                             /mnt/xarfuse/.../IPython/\n\n      aten::copy_         6.34%     338.759ms           0 b  <ipython-input-...>(12): forward\n                                                             /mnt/xarfuse/.../torch/nn\n                                                            

対象の処理において、CPUメモリ使用量が半減しました（日本語訳注：-953.67 Mb　→　-476.84 Mb）。

## 時間パフォーマンスの向上

メモリ消費量に加えて。時間のコストもやや減らせましたが、依然として実行に時間が掛かり過ぎています。

行列をCUDAからCPUにコピーする処理がいかに高コストな処理であるかわかります。

`forward (12)`内の`aten::copy_`は、`mask`をCPUにコピーし、NumPyの`argwhere`関数を使用できるようにしています。

そして`forward(13)`の`aten::copy_`は、配列をテンソルとしてCUDAにコピーし直しています。

代わりに`torch.nonzero()`を使用すれば、これら2つの処理を両方取り除くことができます。

In [ ]:
class MyModule(nn.Module):
    def __init__(self, in_features: int, out_features: int, bias: bool = True):
        super(MyModule, self).__init__()
        self.linear = nn.Linear(in_features, out_features, bias)

    def forward(self, input, mask):
        with profiler.record_function("LINEAR PASS"):
            out = self.linear(input)

        with profiler.record_function("MASK INDICES"):
            threshold = out.sum(axis=1).mean()
            hi_idx = (mask > threshold).nonzero(as_tuple=True)

        return out, hi_idx


model = MyModule(500, 10).cuda()
input = torch.rand(128, 500).cuda()
mask = torch.rand((500, 500, 500), dtype=torch.float).cuda()

# warm-up
model(input, mask)

with profiler.profile(with_stack=True, profile_memory=True) as prof:
    out, idx = model(input, mask)

print(prof.key_averages(group_by_stack_n=5).table(sort_by='self_cpu_time_total', row_limit=5))

"""
(Some columns are omitted)

--------------  ------------  ------------  ------------  ---------------------------------
          Name    Self CPU %      Self CPU  Self CPU Mem   Source Location
--------------  ------------  ------------  ------------  ---------------------------------
      aten::gt        57.17%     129.089ms           0 b  <ipython-input-...>(12): forward
                                                          /mnt/xarfuse/.../torch/nn
                                                          <ipython-input-...>(25): <module>
                                                          /mnt/xarfuse/.../IPython/
                                                          /mnt/xarfuse/.../IPython/

 aten::nonzero        37.38%      84.402ms           0 b  <ipython-input-...>(12): forward
                                                          /mnt/xarfuse/.../torch/nn
                                                          <ipython-input-...>(25): <module>
                                                          /mnt/xarfuse/.../IPython/
                                                          /mnt/xarfuse/.../IPython/

   INDEX SCORE         3.32%       7.491ms    -119.21 Mb  /mnt/xarfuse/.../torch/au
                                                          <ipython-input-...>(10): forward
                                                          /mnt/xarfuse/.../torch/nn
                                                          <ipython-input-...>(25): <module>
                                                          /mnt/xarfuse/.../IPython/

aten::as_strided         0.20%    441.587us          0 b  <ipython-input-...>(12): forward
                                                          /mnt/xarfuse/.../torch/nn
                                                          <ipython-input-...>(25): <module>
                                                          /mnt/xarfuse/.../IPython/
                                                          /mnt/xarfuse/.../IPython/

 aten::nonzero
     _numpy             0.18%     395.602us           0 b  <ipython-input-...>(12): forward
                                                          /mnt/xarfuse/.../torch/nn
                                                          <ipython-input-...>(25): <module>
                                                          /mnt/xarfuse/.../IPython/
                                                          /mnt/xarfuse/.../IPython/
--------------  ------------  ------------  ------------  ---------------------------------
Self CPU time total: 225.801ms

"""

-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  Source Location                                                              
-----------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ---------------------------------------------------------------------------  
          aten::nonzero        94.07%      34.578ms        94.74%      34.826ms      34.826ms           0 b           0 b       2.79 Gb           0 b             1  <ipython-input-7-4175265fd968>(12): forward                                  
                           

'\n(Some columns are omitted)\n\n--------------  ------------  ------------  ------------  ---------------------------------\n          Name    Self CPU %      Self CPU  Self CPU Mem   Source Location\n--------------  ------------  ------------  ------------  ---------------------------------\n      aten::gt        57.17%     129.089ms           0 b  <ipython-input-...>(12): forward\n                                                          /mnt/xarfuse/.../torch/nn\n                                                          <ipython-input-...>(25): <module>\n                                                          /mnt/xarfuse/.../IPython/\n                                                          /mnt/xarfuse/.../IPython/\n\n aten::nonzero        37.38%      84.402ms           0 b  <ipython-input-...>(12): forward\n                                                          /mnt/xarfuse/.../torch/nn\n                                                          <ipython-input-...>(25): <mo

対象の処理において、計算時間が半減しました（日本語訳注：  2.312s 　→　aten::nonzeroが34.578msに）。

## より詳しく学ぶ

プロファイラーを使用してPyTorchのモデル内の時間面、メモリ面のボトルネックを調査する方法を解説しました。

プロファイラーについては、以下の情報もご参考ください。

- [プロファイラーのレシピ](https://pytorch.org/tutorials/recipes/recipes/profiler.html)
- [RPC型ワークロードのプロファイリング](https://pytorch.org/tutorials/recipes/distributed_rpc_profiling.html)
- [プロファイラーAPI ドキュメント](https://pytorch.org/docs/stable/autograd.html?highlight=profiler#profiler)